Based on https://python.langchain.com/docs/tutorials/rag/

OLLAMA embeddings from https://python.langchain.com/docs/integrations/text_embedding/ollama/

Warmup first

In [1]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings

In [11]:
llm = OllamaLLM(model="llama3.2")
llm.invoke("The first man on the moon was ...")

'...Neil Armstrong. He stepped out of the lunar module Eagle and became the first person to set foot on the Moon on July 20, 1969, as part of the Apollo 11 mission.'

Now the real thing

In [2]:
import bs4
# from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
embeddings = OllamaEmbeddings(
    model="llama3.2",
)

In [6]:
# Load, chunk and index the contents of the blog.
# loader = WebBaseLoader("https://www.gutenberg.org/ebooks/3160.txt.utf-8")

loader = WebBaseLoader("https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/create-launch-template.html")
# loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/844/pg844-images.html")
docs = loader.load()
docs

[Document(metadata={'source': 'https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/create-launch-template.html', 'title': 'Create an Amazon EC2 launch template - Amazon Elastic Compute Cloud', 'description': 'Learn how to create Amazon EC2 launch templates.', 'language': 'en-US'}, page_content='\nCreate an Amazon EC2 launch template - Amazon Elastic Compute CloudCreate an Amazon EC2 launch template - Amazon Elastic Compute CloudAWSDocumentationAmazon EC2User GuideCreate a launch template\n                    by specifying parametersCreate a\n                    launch template from an existing launch templateCreate a launch template from\n                    an instanceUse a Systems Manager parameter\n                    instead of an AMI IDCreate an Amazon EC2 launch templateYou can create an Amazon EC2 launch template by specifying your own values for the instance\n            configuration parameters, or by getting the values from an existing launch template or\n            Amazon E

# Careful, below takes long!!

In [7]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

## Option 1

put the docs in the context of the promt

In [12]:
# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()

In [13]:
# Prompt
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [14]:
rag_chain = prompt | llm

In [15]:
rag_chain.invoke({"context": docs, "question":"What is a launch template?"})

'A launch template in Amazon Web Services (AWS) is a template that defines an instance configuration, including the AMI ID, instance type, and tags. It allows users to easily manage and share instances across multiple AWS accounts or regions by providing a reusable set of settings. Launch templates can be used with EC2 Fleets, Spot Fleets, and other services to create instances with consistent configurations.\n\nIn this context, the launch template "TemplateForWebServer" is being created using the `aws ec2 create-launch-template` command, and it includes a Systems Manager parameter specified in place of an AMI ID. This allows the launch template to resolve the actual AMI ID at runtime, rather than hardcoding it into the template itself.\n\nLaunch templates provide several benefits, including:\n\n* Reusability: Launch templates can be shared across multiple AWS accounts or regions, making it easy to manage and deploy consistent instances.\n* Consistency: Launch templates ensure that ins

## Option 2

Pass the documents via the retriever in...

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# we can use a predefined prompt
# prompt = hub.pull("rlm/rag-prompt")

# or cook something up on our own (based on the rag-promt above)
template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
Question: {question}
Context: {context}
Answer:
"""
prompt = ChatPromptTemplate.from_template(template)



def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is a launch template?")

'A launch template is a collection of parameters that you can use to create or modify a launch configuration for an Amazon Elastic Compute Cloud (EC2) instance. It allows you to specify values for various settings such as key pair, network settings, storage, and operating system. Launch templates are used to manage the configuration of your EC2 instances in a centralized way, making it easier to update and maintain your infrastructure.'

In [52]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.\nQuestion: {question}\nnContext: {context}\nAnswer:\n"), additional_kwargs={})])

Let's try without RAG

In [20]:
llm.invoke("What is a launch template?")

'A launch template, also known as a "launch kit" or "sales funnel template," is a pre-built framework that provides a structured approach to launching a new product, service, or campaign. It typically includes a set of essential elements and templates that help entrepreneurs, marketers, and business owners streamline their launch process.\n\nLaunch templates usually include:\n\n1. Branding and positioning materials (e.g., logos, taglines, images)\n2. Sales page template with copywriting and design suggestions\n3. Email marketing campaigns (e.g., welcome series, promotional emails)\n4. Social media content calendar\n5. Paid advertising ads (e.g., Facebook, Google Ads)\n6. Opt-in page templates for capturing leads\n7. Landing page templates for specific sales funnels\n\nHaving a launch template can help you:\n\n1. Save time and effort in planning and executing your launch\n2. Ensure consistency in branding and messaging across all channels\n3. Test and optimize different marketing strate